<a href="https://colab.research.google.com/github/jojowaranyucareer-ctrl/Weatherwise_Waranyu.B/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [56]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

In [83]:
import os

# Configure your provider
os.environ['HANDS_ON_AI_SERVER'] = 'https://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'llama3.2'
os.environ['HANDS_ON_AI_API_KEY'] = 'e6903b35e08f4227b3a4cbee5b836408'

# Now use HandsOnAI
from hands_on_ai.chat import get_response
print(get_response("As a current weather in Australia, Perth today is 30 degrees celcius, with uv index of, and partial clouds, Can i play outddoor basketball?"))

Playing outdoor basketball on a day like that might not be the best idea.

With a temperature of 30°C (86°F), it's definitely warm enough to make outdoor activities uncomfortable. The UV index is also quite high, which increases your risk of sunburn and skin damage.

Additionally, with partial clouds, while there isn't a thick layer of cloud cover to block out all the sun's rays, some shade might still be available. However, if you're going to be playing basketball outside during peak sun hours (usually between 10am-4pm), it's likely that the sun will be shining brightly enough to cause discomfort.

Considering these factors, I would advise against playing outdoor basketball on this day in Perth. Would you like some suggestions for indoor sports facilities or alternative activities?


## 📦 Setup and Configuration
Import required packages and setup environment.

In [81]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from hands_on_ai.chat import get_response

# Add any other setup code here

# Core libraries
import os
import re
from typing import Dict, Any, List, Optional, Tuple, Union
import requests
import matplotlib.pyplot as plt

os.environ["OPENWEATHER_API_KEY"] = "3a10e176bbe73766db6e4d09a4bd96d0"  # Demo key, get your own at https://openweathermap.org/api

## 🌤️ Weather Data Functions

In [82]:
# Define get_weather_data() function here
from typing import TypedDict

class ParsedWeather(TypedDict, total=False):
	location: str
	forecast_days: int
	error: Optional[str]
	current: Dict[str, Any]
	daily: List[Dict[str, Any]]


def _normalize_fmw_json(model_or_dict: Any) -> Dict[str, Any]:
	"""
	Convert fetch_my_weather WeatherResponse (Pydantic) or raw dict to a plain dict.
	This keeps only the keys we care about for the assignment to simplify downstream code.
	"""
	# If it's a pydantic model, use model_dump if available
	if hasattr(model_or_dict, "model_dump"):
		data = model_or_dict.model_dump()  # type: ignore[attr-defined]
	else:
		data = dict(model_or_dict)
	return data

def openWeather_getWeather(city: str, units: str = "metric"):
	api_key = os.getenv("OPENWEATHER_API_KEY")
	if not api_key:
		return None
	try:
		CITY = city
		UNITS = units  # metric for °C or "imperial" for °F
		# Step 1: Call the 5-day forecast API
		url = f"https://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={api_key}&units={UNITS}"
		response = requests.get(url)
		data = response.json()
		return data
	except Exception as e:
		print(f"Error fetching weather data: {e}")
		return None


def get_weather_data(city: str, forecast_days: int = 5):
	"""
	Retrieve weather data for a specified location.

	Args:
		location (str): City or location name
		forecast_days (int): Number of days to forecast (1-5)

	Returns:
		dict: Weather data including current conditions and forecast
	"""
	try:
		response = openWeather_getWeather(city=city, units="metric")
		return response
	except Exception as e:
		response["error"] = f"Failed to fetch weather data: {e}"
		return response

## 📊 Visualisation Functions

In [68]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
from typing import List, Dict, Any

def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels

def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps


def create_temperature_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of temperature data using average daily temperatures.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("list", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        daily_temps = get_temps(weather_data)
        # Slice the data to include only the requested number of days
        sliced_dates = list(daily_temps.keys())[:num_days]
        sliced_temps = [sum(daily_temps[date]) / len(daily_temps[date]) if daily_temps[date] else 0 for date in sliced_dates]


        fig, ax = plt.subplots()
        ax.plot(sliced_dates, sliced_temps, marker='o', label='Average Temp (°C)')
        ax.set_title(f"Daily Average Temperature Forecast for {num_days} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None

In [89]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
from typing import List, Dict, Any
from collections import defaultdict


def _extract_day_labels(daily: List[Dict[str, Any]]) -> List[str]:
        labels: List[str] = []
        for d in daily:
                date_str = d.get("date") or d.get("astronomy", [{}])[0].get("sunrise")
                try:
                        labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
                except Exception:
                        labels.append(str(date_str))
        return labels

def get_temps(data):
        daily_temps = {}

        for item in data["list"]:
                dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
                date_str = dt_txt.split(" ")[0] # Just the date part
                temp = item["main"]["temp"]

                if date_str not in daily_temps:
                        daily_temps[date_str] = []

                daily_temps[date_str].append(temp)

        return daily_temps


def create_temperature_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of temperature data using average daily temperatures.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """

        # get daily forecast data
        daily = weather_data.get("list", [])
        if not daily:
                print("No daily forecast data available to plot temperatures.")
                return None

        daily_temps = get_temps(weather_data)
        # Slice the data to include only the requested number of days
        sliced_dates = list(daily_temps.keys())[:num_days]
        sliced_temps = [sum(daily_temps[date]) / len(daily_temps[date]) if daily_temps[date] else 0 for date in sliced_dates]


        fig, ax = plt.subplots()
        ax.plot(sliced_dates, sliced_temps, marker='o', label='Average Temp (°C)')
        ax.set_title(f"Daily Average Temperature Forecast for {num_days} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Temperature (°C)")
        ax.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None


def create_precipitation_visualisation(weather_data: Dict[str, Any], num_days: int = 5, output_type: str = 'display'):
        """
        Create visualisation of precipitation data.

        Args:
                weather_data (dict): The processed weather data
                num_days (int): The number of days to display in the chart.
                output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

        Returns:
                If output_type is 'figure', returns the matplotlib figure object
                Otherwise, displays the visualisation in the notebook
        """
        # OpenWeatherMap provides data in 3-hour intervals under the 'list' key
        forecast_list = weather_data.get("list", [])
        if not forecast_list:
                print("No forecast data available to plot precipitation.")
                return None

        daily_precipitation = defaultdict(float)
        date_labels = []

        for item in forecast_list:
            dt_txt = item["dt_txt"] # e.g., '2025-10-16 03:00:00'
            date_str = dt_txt.split(" ")[0] # Just the date part

            # Add date to labels if not already present (ensures unique daily labels)
            if date_str not in date_labels:
                date_labels.append(date_str)

            # OpenWeatherMap uses 'rain' or 'snow' keys with a '3h' sub-key
            # Total precipitation is the sum of rain and snow in the 3-hour period
            precip_3h = item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0)
            daily_precipitation[date_str] += precip_3h # Accumulate precipitation for the day


        # Sort the daily precipitation data by date
        sorted_dates = sorted(daily_precipitation.keys())
        sorted_precipitation_totals = [daily_precipitation[date] for date in sorted_dates]


        # Format date labels for the plot
        formatted_labels: List[str] = []
        for date_str in sorted_dates:
            try:
                formatted_labels.append(datetime.strptime(date_str, "%Y-%m-%d").strftime("%a %d"))
            except Exception:
                formatted_labels.append(date_str)


        fig, ax = plt.subplots()
        # Limit to the number of days specified by the user
        num_days_to_plot = min(num_days, len(sorted_dates))
        ax.bar(formatted_labels[:num_days_to_plot], sorted_precipitation_totals[:num_days_to_plot], color="#4e79a7")
        ax.set_title(f"Daily Total Precipitation Forecast for {num_days_to_plot} Days")
        ax.set_xlabel("Day")
        ax.set_ylabel("Precipitation (mm)")
        plt.tight_layout()

        if output_type == 'figure':
                return fig
        else:
                plt.show()
                return None

## 🤖 Natural Language Processing

In [92]:
from typing import Dict, Any
from hands_on_ai.chat import get_response

def parse_weather_question(question: str) -> Dict[str, Any]:
        """
        Parse a natural language weather question.

        Args:
                question (str): User's weather-related question

        Returns:
                dict: Extracted information including location, time period, and weather attribute
        """
        if not question or not isinstance(question, str):
                return {"error": "Invalid question provided."}

        # Basic implementation to return a dummy structure
        return {
                "raw": question,
                "location": None,
                "temperature": None,
                "uv_index": None,
                "weather_condition": None
        }

def get_ai_weather_response(weather_data: Dict[str, Any], question: str) -> str:
    """
    Get an AI-generated response to a weather question based on provided weather data.

    Args:
        weather_data (dict): The processed weather data.
        question (str): The user's weather-related question.

    Returns:
        str: The AI's natural language response.
    """
    # Combine weather data into a string to provide context to the AI
    weather_context = f"Current weather data: "
    if weather_data and weather_data.get("list"):
        current = weather_data["list"][0]
        weather_context += f"Temperature: {current['main']['temp']:.1f}°C, Conditions: {current['weather'][0]['description']}, Humidity: {current['main']['humidity']}%, Wind: {current['wind']['speed']} m/s."

        if len(weather_data["list"]) > 8: # Add tomorrow's forecast if available
            tomorrow_data_points = weather_data["list"][8:16]
            if tomorrow_data_points:
                tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]
                tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0
                tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)
                tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                weather_context += f" Tomorrow ({tomorrow_date}): Average Temperature: {average_temp_tomorrow:.2f}°C, Precipitation: {tomorrow_precipitation:.2f} mm, Conditions: {tomorrow_weather_description}."
    else:
        weather_context += "Weather data not available."


    full_question = f"{weather_context}\nBased on this weather information, {question}"

    try:
        # Use the get_response function from hands_on_ai
        ai_response = get_response(full_question)
        return ai_response
    except Exception as e:
        return f"Error getting AI response: {e}"

## 🧭 User Interface

In [105]:
# Define menu functions using pyinputplus or ipywidgets here
import ipywidgets as widgets
from IPython.display import display

# Initialize global variables
_LAST_WEATHER = None
_LAST_LOCATION = None
_LAST_FORECAST_DAYS = 5

def create_weather_ui():
    """
    Creates and returns the WeatherWise user interface.
    """
    # Create input widgets
    location_input = widgets.Text(
        value=_LAST_LOCATION if _LAST_LOCATION else '',
        placeholder='Enter location (e.g., Perth)',
        description='Location:',
        disabled=False
    )

    days_input = widgets.IntText(
        value=_LAST_FORECAST_DAYS,
        description='Days (1-6):',
        disabled=False,
        min=1,
        max=6,
        layout=widgets.Layout(description_width='250 px') # Adjust description width
    )

    # New input field for the question
    question_input = widgets.Text(
        value='',
        placeholder='Ask a weather question (e.g., Is it going to rain tomorrow?)',
        description='Question:',
        disabled=False
    )


    # Create action buttons
    button_layout = widgets.Layout(width='250px') # Define a layout for buttons

    fetch_button = widgets.Button(description="Fetch Weather Forecast", layout=button_layout)
    temp_chart_button = widgets.Button(description="Show Temperature Chart", layout=button_layout)
    precip_chart_button = widgets.Button(description="Show Precipitation Chart", layout=button_layout)
    should_go_outside_button = widgets.Button(description="Should we go outside tomorrow?", layout=button_layout)
    ask_question_button = widgets.Button(description="Ask Weather Question", layout=button_layout) # New button
    quit_button = widgets.Button(description="Quit", layout=button_layout)

    # Create an output widget to display results
    output_area = widgets.Output()

    # Arrange widgets in a layout
    input_widgets = widgets.VBox([location_input, days_input, question_input]) # Added question_input
    button_widgets = widgets.VBox([fetch_button, temp_chart_button, precip_chart_button, should_go_outside_button, ask_question_button, quit_button]) # Added ask_question_button

    ui = widgets.VBox([
        widgets.Label("Greetings! I'm your WeatherWise assistant, ready to bring you the latest forecast."),
        input_widgets,
        button_widgets,
        output_area
    ])

    # Define button click handlers
    def on_fetch_button_clicked(b):
        with output_area:
            output_area.clear_output()
            global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
            loc = location_input.value
            if not loc:
                print("Please enter a location.")
                return
            # Fetch only today's weather for this button
            days = 6  # Fetch 6 days to potentially use for summary or other future features
            data = get_weather_data(loc, forecast_days=days)

            _LAST_WEATHER = data
            _LAST_LOCATION = loc
            _LAST_FORECAST_DAYS = days

            if data and data.get("list"):
                daily_temps = get_temps(data)
                if daily_temps:
                    # Get current temperature (using the first available data point as a proxy)
                    current_temp = data["list"][0]["main"]["temp"]
                    current_description = data["list"][0]["weather"][0]["description"]
                    humidity = data["list"][0]["main"]["humidity"]
                    wind_speed = data["list"][0]["wind"]["speed"]

                    print(f"Weather for {loc}:")
                    print(f"Currently: {current_temp:.1f}°C with {current_description}, Humidity: {humidity}%, Wind: {wind_speed} m/s")

                    # Get average temperature for the first day (today)
                    first_day_date = list(daily_temps.keys())[0]
                    temps = daily_temps[first_day_date]
                    average_temp = sum(temps) / len(temps) if temps else 0

                    print(f"Average temperature for Today ({first_day_date}): {average_temp:.2f}°C")

                    # Add conditional messages based on average temperature
                    if 28 <= average_temp <= 32:
                        print("Advice: It is quite hot today. Please wear cool clothing and use sunscreen.")
                    elif 22 <= average_temp <= 27:
                        print("Advice: Today has good weather.")
                    elif 17 <= average_temp <= 21:
                        print("Advice: It is quite cold today. Please wear warm clothes if you go out.")
                    elif average_temp < 17:
                        print("Advice: It is freezing! Stay home and turn on the heater.")
                    elif average_temp > 32: # Corrected condition for "burning"
                        print("Advice: It is burning hot! Stay home and turn on the air conditioning.")
                else:
                     print("Could not retrieve daily temperature data for the specified location.")


            elif data and data.get("message"):
                 print(f"Error fetching weather data: {data['message']}")
            else:
                 print("Could not retrieve weather data for the specified location.")


    def on_temp_chart_button_clicked(b):
        with output_area:
            output_area.clear_output()
            global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
            loc = location_input.value
            if not loc:
                print("Please enter a location.")
                return
            if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
                print(f"Fetching weather data for {loc}...")
                days = 6
                data = get_weather_data(loc, forecast_days=days)
                _LAST_WEATHER = data
                _LAST_LOCATION = loc
                _LAST_FORECAST_DAYS = days
                if not data or data.get("message"):
                    print("Could not retrieve weather data. Please check location.")
                    return

            if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
                print("Weather data is not available to create chart. Please fetch weather first.")
                return

            num_days_chart = days_input.value
            create_temperature_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')

    def on_precip_chart_button_clicked(b):
        with output_area:
            output_area.clear_output()
            global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS # Moved global declaration to the top
            loc = location_input.value
            if not loc:
                print("Please enter a location.")
                return
            if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
                print(f"Fetching weather data for {loc}...")
                days = 6
                data = get_weather_data(loc, forecast_days=days)
                _LAST_WEATHER = data
                _LAST_LOCATION = loc
                _LAST_FORECAST_DAYS = days
                if not data or data.get("message"):
                    print("Could not retrieve weather data. Please check location.")
                    return

            if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
                print("Weather data is not available to create chart. Please fetch weather first.")
                return

            # Get the number of days from the input widget
            num_days_chart = days_input.value
            create_precipitation_visualisation(_LAST_WEATHER, num_days=num_days_chart, output_type='display')


    def on_should_go_outside_button_clicked(b):
        with output_area:
            output_area.clear_output()
            global _LAST_WEATHER, _LAST_LOCATION, _LAST_FORECAST_DAYS
            loc = location_input.value
            if not loc:
                print("Please enter a location.")
                return

            if not _LAST_WEATHER or _LAST_LOCATION != loc: # Fetch if no data or location changed
                print(f"Fetching weather data for {loc}...")
                days = 6
                data = get_weather_data(loc, forecast_days=days)
                _LAST_WEATHER = data
                _LAST_LOCATION = loc
                _LAST_FORECAST_DAYS = days
                if not data or data.get("message"):
                    print("Could not retrieve weather data. Please check location.")
                    return

            if not _LAST_WEATHER or not _LAST_WEATHER.get("list"):
                print("Weather data is not available for tomorrow's forecast. Please fetch weather first.")
                return

            # Get tomorrow's data - assuming the first day in the list is today, the second is tomorrow
            if len(_LAST_WEATHER.get("list", [])) > 8: # OpenWeatherMap provides data in 3-hour intervals, so need at least 8*3 = 24 hours for tomorrow
                tomorrow_data_points = _LAST_WEATHER["list"][8:16] # Assuming data points 8-15 cover tomorrow
                if tomorrow_data_points:
                    # Calculate average temperature for tomorrow
                    tomorrow_temps = [item["main"]["temp"] for item in tomorrow_data_points]
                    average_temp_tomorrow = sum(tomorrow_temps) / len(tomorrow_temps) if tomorrow_temps else 0

                    # Calculate total precipitation for tomorrow (summing up 3-hour intervals)
                    # OpenWeatherMap's forecast doesn't always have 'rain' or 'snow' keys, need to check
                    tomorrow_precipitation = sum(item.get("rain", {}).get("3h", 0) + item.get("snow", {}).get("3h", 0) for item in tomorrow_data_points)

                    # Get wind speed and description for tomorrow (using the first data point for simplicity)
                    tomorrow_wind_speed = tomorrow_data_points[0]["wind"]["speed"]
                    tomorrow_weather_description = tomorrow_data_points[0]["weather"][0]["description"]

                    tomorrow_date = tomorrow_data_points[0]["dt_txt"].split(" ")[0]

                    print(f"Tomorrow's Weather ({tomorrow_date}) for {loc}:")
                    print(f"Average Temperature: {average_temp_tomorrow:.2f}°C")
                    print(f"Total Precipitation: {tomorrow_precipitation:.2f} mm")
                    print(f"Wind Speed: {tomorrow_wind_speed:.2f} m/s")
                    print(f"Conditions: {tomorrow_weather_description}")


                    # Provide advice based on tomorrow's weather
                    print("\nShould we go outside tomorrow?")
                    if tomorrow_precipitation > 0.5: # Threshold for significant rain
                        print("Advice: It looks like there will be significant rain tomorrow. It might be best to stay inside or be prepared for wet conditions.")
                    elif average_temp_tomorrow < 10: # Threshold for cold
                         print("Advice: It will be quite cold tomorrow. Bundle up if you go outside!")
                    elif average_temp_tomorrow > 30: # Threshold for hot
                         print("Advice: It will be quite hot tomorrow. Stay hydrated and seek shade if you go outside.")
                    else:
                        print("Advice: The weather looks good for going outside tomorrow!")

                else:
                    print("Could not retrieve detailed forecast data for tomorrow.")

            else:
                 print("Insufficient forecast data to determine tomorrow's weather.")

    def on_ask_question_button_clicked(b):
        with output_area:
            output_area.clear_output()
            global _LAST_WEATHER, _LAST_LOCATION
            question = question_input.value
            loc = location_input.value

            if not loc:
                print("Please enter a location.")
                return

            if not question:
                print("Please enter a question.")
                return

            if not _LAST_WEATHER or _LAST_LOCATION != loc:
                print(f"Fetching weather data for {loc} before answering...")
                days = 6
                data = get_weather_data(loc, forecast_days=days)
                _LAST_WEATHER = data
                _LAST_LOCATION = loc
                _LAST_FORECAST_DAYS = days
                if not data or data.get("message"):
                    print("Could not retrieve weather data. Cannot answer question.")
                    return

            print("Asking AI...")
            # Call the new function in the Natural Language Processing section
            ai_response = get_ai_weather_response(_LAST_WEATHER, question)
            print("\nAI Response:")
            print(ai_response)


    def on_quit_button_clicked(b):
        with output_area:
            output_area.clear_output()
            print("Goodbye!")
        # In a real application, you might want to stop execution here
        # For a notebook, simply printing a message is sufficient


    # Link buttons to handlers
    fetch_button.on_click(on_fetch_button_clicked)
    temp_chart_button.on_click(on_temp_chart_button_clicked)
    # summary_button.on_click(on_summary_button_clicked) # Removed
    precip_chart_button.on_click(on_precip_chart_button_clicked)
    should_go_outside_button.on_click(on_should_go_outside_button_clicked)
    ask_question_button.on_click(on_ask_question_button_clicked) # Link new button
    quit_button.on_click(on_quit_button_clicked)

    return ui

# The UI will now be displayed by calling create_weather_ui() in the Testing and Examples section

## 🧩 Main Application Logic

In [99]:
# Tie everything together here

def generate_weather_response(parsed_question: Dict[str, Any], weather_data: Dict[str, Any]) -> str:
	"""
	Generate a natural language response to a weather question.

	Args:
		parsed_question (dict): Parsed question data
		weather_data (dict): Weather data

	Returns:
		str: Natural language response
	"""
	if parsed_question.get("error"):
		return parsed_question["error"]
	if weather_data.get("error"):
		return weather_data["error"]

	location = weather_data.get("location") or parsed_question.get("location") or "the selected location"
	current = weather_data.get("current", {})
	daily = weather_data.get("daily", [])

	attribute = parsed_question.get("attribute")
	time_spec = parsed_question.get("time")

	def format_day(d: Dict[str, Any]) -> str:
		date = d.get("date", "")
		desc = (d.get("hourly", [{}])[0].get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		return f"{date} ({desc})"

	if attribute == "temperature":
		# If today requested or unspecified, use current
		if time_spec in (None, 0):
			t = current.get("temp_C")
			feels = current.get("FeelsLikeC") or current.get("FeelsLikeC")
			desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
			if t is not None:
				return f"Right now in {location}: {t}°C, {desc}. Feels like {feels}°C."
		# Otherwise use daily forecast
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			return (
				f"In {location} on {d.get('date')}: min {d.get('mintempC')}°C, max {d.get('maxtempC')}°C."
			)
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('mintempC')}–{d.get('maxtempC')}°C")
			if parts:
				return f"Temperature forecast for {location}: " + "; ".join(parts)

	if attribute == "precipitation":
		if isinstance(time_spec, int) and 0 <= time_spec < len(daily):
			d = daily[time_spec]
			mm = d.get("totalPrecipMM")
			return f"Expected precipitation in {location} on {d.get('date')}: {mm} mm."
		if time_spec in (None, 0):
			# Try current precipitation
			precip_mm = current.get("precipMM")
			if precip_mm is not None:
				return f"Right now in {location}, precipitation is {precip_mm} mm."
		if isinstance(time_spec, slice):
			parts = []
			for d in daily[time_spec]:
				parts.append(f"{d.get('date')}: {d.get('totalPrecipMM', 0)} mm")
			if parts:
				return f"Precipitation forecast for {location}: " + "; ".join(parts)

	if attribute == "wind":
		if time_spec in (None, 0):
			spd = current.get("windspeedKmph")
			dirp = current.get("winddir16Point")
			if spd is not None:
				return f"Current wind in {location}: {spd} km/h {dirp or ''}."

	# Fallback generic summary
	if current:
		desc = (current.get("weatherDesc") or [{"value": ""}])[0].get("value", "")
		t = current.get("temp_C")
		return f"Currently in {location}: {t}°C, {desc}."

	return "I'm unable to summarise the weather from the provided data."

## 🧪 Testing and Examples

In [106]:
# Include sample input/output for each function

# Display the UI
weather_ui = create_weather_ui()
display(weather_ui)

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.